# Fallstudie Tablet GmbH

Finale Datei zur Abgabe am Ende des Projektes </br>


In [19]:
import pandas as pd
import numpy as np

### Datenaufbereitung

Der erste Schritt ist die Aufarbeitung der Datengrundlage, um eine Konsistente Nutzung dieser sicherzustellen. Hierzu werden folgende Punkte betrachtet:
<ul>
    <li>
        Spaltennamen konsistent
        <ul>
            <li>Englisch</li>
            <li>klein geschrieben</li>
            <li>keine Leerzeichen</li>
            <li>gleiches Prinzip bei Bezeichnung</li>
        </ul>
    </li>
    <li>Zum Wert passende Datentypen</li>
    <li>Hinzufügen weiterer Spalten</li>
    <li>Zusammenfügen von Daten verschiedener Tabellen</li>
</ul>

Für die Tabelle mit den Preisen müssen die Spaltennamen angepasst werden. Außerdem müssen die Werte der Jahresspalte bereinigt und der Datentyp angepasst werden.

In [21]:
df_price = (
    pd.read_excel("./data/Preise.xlsx") #./../../data/Preise.xlsx   - Pfad für Sandbox
        .rename(columns={"Jahr": "year",
                         "preis": "price"})
        .assign(year=lambda _data: pd.to_numeric(_data["year"].str.strip('"')))
        .replace([20022, "Tablet Basics"],[2022, "Tablet Basic"])
)

In der Downtime Tabelle müssen nur die Spaltennamen angepasst werden.

In [22]:
df_dt = (
    pd.read_excel("./data/Downtime.xlsx")
        .rename(columns={"Unnamed: 0": "year",
                         "Region A": "a",
                         "Region B": "b",
                         "Region C": "c",
                         "Region D": "d"})
)

In der Kundendaten Tabelle müssen auch nur die Spaltennamen angepasst werden. Eine Prüfung hat ergeben, dass es in dieser Tabelle keine Duplikate und keine Null Werte gibt.

In [23]:
df_crm = (
    pd.read_excel("./data/CRM_data.xlsx")
        .rename(columns={"id": "transaction_id",
                         "vorname": "firstname",
                         "nachname": "lastname"})
)

Bei den Tabellen mit den Transaktionsdaten können wir bei allen Tabellen gleich vorgehen. Daher macht es hier Sinn eine Funktion für die Aufarbeitung zu schreiben und diese für jede Datei aufzurufen. Hierbei müssen zunächst die Spaltennamen angepasst werden. Danach werden neue Spalten für das Jahr, den Monat und die Woche hinzugefügt, um bei der Analyse eine einfachere Einteilung in Zeitabschnitte zu bieten. Außerdem werden die Werte der Kosten auf zwei Nachkommastellen gerundet und es wird eine neue Spalte, welche immer die Kosten enthält, hinzugefügt. Danach werden noch Spalten wíe die Gesamtkosten der Transaktion, Umsatz und Gewinn hinzugefügt. Außerdem werden der geltende Preis für das Produkt und die Kundenbewertung der Transaktion der Tabelle hinzugefügt.

In [26]:
def read_regio_data(region, kind): 
    return (pd.read_excel(f"./data/Region{region}_{kind}.xlsx")
                .rename(columns={"Tablet Basic": "basic",
                                "Tablet Mini": "mini",
                                "Tablet Pro": "pro",
                                "kosten_mini": "cost_mini",
                                "kosten_basic": "cost_basic",
                                "kosten_pro": "cost_pro",
                                "PRODUKT": "product"})
                .assign(year=lambda _data: _data["date"].dt.year,
                        month=lambda _data: _data["date"].dt.month,
                        week=lambda _data: _data["date"].dt.isocalendar().week,
                        cost_basic=lambda _data: np.round(_data["cost_basic"], 2),
                        cost_mini=lambda _data: np.round(_data["cost_mini"], 2),
                        cost_pro=lambda _data: np.round(_data["cost_pro"], 2),
                        cost_per_unit=lambda _data: _data["cost_basic"].fillna(0) + _data["cost_mini"].fillna(0) + _data["cost_pro"].fillna(0),
                        cost=lambda _data: _data["cost_per_unit"] * _data["amount"]
                        )
                .merge(df_price, on=["year", "product"])
                .assign(revenue=lambda _data: _data["price"] * _data["amount"],
                        profit_per_unit=lambda _data: _data["price"] - _data["cost_per_unit"],
                        profit=lambda _data: _data["profit_per_unit"] * _data["amount"])
                .merge(df_crm[["customer_rating", "transaction_id"]], on="transaction_id"))

df_regAo = read_regio_data("A","online")
df_regAp = read_regio_data("A","phone")
df_regAs = read_regio_data("A","store")
df_regBo = read_regio_data("B","online")
df_regBp = read_regio_data("B","phone")
df_regBs = read_regio_data("B","store")
df_regCo = read_regio_data("C","online")
df_regCp = read_regio_data("C","phone")
df_regCs = read_regio_data("C","store")
df_regDo = read_regio_data("D","online")
df_regDp = read_regio_data("D","phone")
df_regDs = read_regio_data("D","store")